In [1]:
!pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [2]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

In [3]:
import numpy as np
import pandas as pd

def create_dataframe(output, label):
    df = pd.read_excel("../../../data.xlsx")
    x1, x2, x3, x4, x5  = df['x1'], df['x2'], df['x3'], df['x4'], df['x5']

    input = np.vstack([x1, x2, x3, x4, x5 ]).T
    output = np.array(df[output])
    return output, input

output, input = create_dataframe(output="y1", label="ph")

In [4]:
import tensorflow as tf
import numpy as np
from keras import regularizers
from keras import initializers
import levenberg_marquardt as lm

# layers, neurons
class ShuffleArchitecture:
    def __init__(self, input_size, hidden_sizes, output_size, act_h, act_o, param_reg):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.act_h = act_h
        self.act_o = act_o
        self.regularizer = regularizers.L2(param_reg)
        self.initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

    def set_architecture(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(self.hidden_sizes[0],
                        input_shape=(self.input_size,),
                        activation=self.act_h,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,                        
                        ))  # input layer
        
        for size in self.hidden_sizes[1:]:  # hidden layers
            self.model.add(tf.keras.layers.Dense(size,
                            activation=self.act_h,
                            kernel_regularizer=self.regularizer,
                            kernel_initializer=self.initializer,  
                        ))

        self.model.add(tf.keras.layers.Dense(self.output_size,
                        activation=self.act_o,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,  
                        ))  # output layer

    def create_model(self, _learning_rate):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=_learning_rate),
            loss=tf.keras.losses.MeanSquaredError())

        self.lm_model = lm.ModelWrapper(
            tf.keras.models.clone_model(self.model))

        self.lm_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=_learning_rate),
            loss=lm.MeanSquaredError())
        return(self.lm_model)

2024-02-27 23:21:34.608843: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-27 23:21:36.869632: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-27 23:21:36.869756: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-27 23:21:37.207278: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-27 23:21:37.926658: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-27 23:21:37.928637: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error


class TrainWithSmallDataset:
    def __init__(self, batch_size=1000):
        self.batch_size = batch_size
        self.betters = []
    
    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)

    def split_dataset(self, input, output):
      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)
 
 
    
    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)

          metrics = {
                          'r2': r2,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics

In [6]:
from itertools import product
import pickle

class Tester:
  def __init__(self, input, output, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input, self.output = input, output
  
  def setArchitecure(self, trainer, _hidden_sizes, _pg, _lr):
    shuffler = ShuffleArchitecture(input_size=5,
                                    hidden_sizes=_hidden_sizes,
                                    output_size=1,
                                    act_h='tanh',
                                    act_o='linear',
                                    param_reg=_pg)
    shuffler.set_architecture()    
    trainer.lm_model = shuffler.create_model(_lr)

  def Train(self, trainer, epochs=1000):
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path) 

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)


  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ [{idx}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] >= boundarie): # should be <= to descending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

# Init
  def Loop(self, sort_by, boundarie, hidden_sizes, regularizers, learning_rate):
    trainer = TrainWithSmallDataset()

    for count, (hidden_size, reg, lr) in enumerate(product(hidden_sizes, regularizers, learning_rate), start=1):
      header =  f"Hidden Size={hidden_size}, regularizer={reg}, learning_rate={lr}"
      print(f"Testando combinacao{count}: {header}")
      self.setArchitecure(trainer, hidden_size, reg, lr)
      for j in range(self.dataset_run_times):
        trainer.split_dataset(self.input, self.output)
        if (self.LoopWeights(sort_by, boundarie, trainer, f"{count}_{j}") == True):
          self.SaveDataset(trainer, f"dataset_{count}_{j}")
          self.DisplayBetterResults(sort_by, header, f"{count}_{j}")
        self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, header, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    path = f'../results/metrics_{dataset}'
    df.to_excel(f"{path}.xlsx", index=True)
    print(f"DataFrame salvo em {path}")
    with open(f"{path}.txt", 'w') as arquivo:
      arquivo.write(header)

In [7]:
tester = Tester(input, output, run_times=25, dataset_run_times=10)

In [8]:
tester.Loop(sort_by='r2',
            boundarie=0.8,
            hidden_sizes = [[15, 7], [30, 15], [30, 15, 7]],
            regularizers=[0.02, 0.1],
            learning_rate=[0.02, 0.1])

Testando combinacao1: Hidden Size=[15, 7], regularizer=0.02, learning_rate=0.02
+++++++++++ [1_0] | 1 ++++++++++++++++++
Elapsed time:  1.2978896949999807
Stopped at epoch:  15
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 2 ++++++++++++++++++
Elapsed time:  0.4521685370000341
Stopped at epoch:  10
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_0] | 3 ++++++++++++++++++
Elapsed time:  1.5966898550000224
Stopped at epoch:  34
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 4 ++++++++++++++++++
Elapsed time:  0.7010854699999527
Stopped at epoch:  14
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_0] | 5 ++++++++++++++++++
Elapsed time:  0.2936890129999483
Stopped at epoch:  6
1/1 [==============================] - 0s 26ms/step
+++++++++++ [1_0] | 6 ++++++++++++++++++
Elapsed time:  1.3468096880000076
Stopped at epoch:  25
1/1 [==============================] - 0s 16ms/step
+++++++++++ [1_0] | 7 +++++

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_6_2_7,0.833300,0.557204,-0.312221,0.410996,0.031186,0.113731,0.076658,0.096284
model_6_2_6,0.839255,0.585601,-0.263410,0.442821,0.030072,0.106437,0.073807,0.091082
model_6_2_5,0.845334,0.614871,-0.204444,0.477085,0.028934,0.098919,0.070362,0.085481
model_6_2_0,0.845566,0.688464,0.093794,0.588458,0.028891,0.080017,0.052939,0.067275
model_6_2_1,0.851582,0.675456,0.101981,0.579015,0.027766,0.083358,0.052461,0.068818
model_6_2_4,0.851635,0.641189,-0.110297,0.514813,0.027756,0.092159,0.064862,0.079313
model_6_2_3,0.855641,0.659987,-0.010827,0.547177,0.027006,0.087331,0.059051,0.074023
model_6_2_2,0.856472,0.674398,0.061511,0.571329,0.026851,0.083630,0.054825,0.070075


DataFrame salvo em ../results/metrics_6_2
+++++++++++ [6_3] | 1 ++++++++++++++++++
Elapsed time:  0.3239499839999098
Stopped at epoch:  6
1/1 [==============================] - 0s 22ms/step
+++++++++++ [6_3] | 2 ++++++++++++++++++
Elapsed time:  0.5022469989999081
Stopped at epoch:  6
1/1 [==============================] - 0s 83ms/step
+++++++++++ [6_3] | 3 ++++++++++++++++++
Elapsed time:  0.37995464400000856
Stopped at epoch:  6
1/1 [==============================] - 0s 28ms/step
+++++++++++ [6_3] | 4 ++++++++++++++++++
Elapsed time:  0.33474755199995343
Stopped at epoch:  6
1/1 [==============================] - 0s 24ms/step
+++++++++++ [6_3] | 5 ++++++++++++++++++
Elapsed time:  0.36054291000004923
Stopped at epoch:  6
1/1 [==============================] - 0s 20ms/step
+++++++++++ [6_3] | 6 ++++++++++++++++++
Elapsed time:  0.3189520010000706
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [6_3] | 7 ++++++++++++++++++
Elapsed time:  0.330069490

2024-02-27 23:48:03.077932: W tensorflow/core/data/root_dataset.cc:342] Optimization loop failed: CANCELLED: Operation was cancelled


Elapsed time:  0.6697499110000535
Stopped at epoch:  6
1/1 [==============================] - 0s 17ms/step
+++++++++++ [10_4] | 18 ++++++++++++++++++
Elapsed time:  0.7500572230001126
Stopped at epoch:  6
1/1 [==============================] - 0s 20ms/step
+++++++++++ [10_4] | 19 ++++++++++++++++++
Elapsed time:  8.850331904000086
Stopped at epoch:  122
1/1 [==============================] - 0s 28ms/step
+++++++++++ [10_4] | 20 ++++++++++++++++++
Elapsed time:  0.39035315999990416
Stopped at epoch:  6
1/1 [==============================] - 0s 17ms/step
+++++++++++ [10_4] | 21 ++++++++++++++++++
Elapsed time:  0.34215799499997956
Stopped at epoch:  6
1/1 [==============================] - 0s 22ms/step
+++++++++++ [10_4] | 22 ++++++++++++++++++
Elapsed time:  0.5239415689998168
Stopped at epoch:  6
1/1 [==============================] - 0s 21ms/step
+++++++++++ [10_4] | 23 ++++++++++++++++++
Elapsed time:  0.5968348910000714
Stopped at epoch:  6
1/1 [==============================] - 0s 

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_10_6_0,0.801482,-0.187183,-0.321990,-0.189294,0.037138,0.127741,0.097960,0.113726
model_10_6_1,0.803349,-0.180393,-0.346785,-0.194292,0.036789,0.127010,0.099797,0.114204
model_10_6_2,0.804853,-0.174400,-0.367894,-0.198418,0.036508,0.126365,0.101361,0.114599
model_10_6_3,0.806126,-0.168877,-0.385115,-0.201408,0.036269,0.125771,0.102637,0.114885
model_10_6_4,0.807201,-0.164006,-0.398993,-0.203566,0.036068,0.125247,0.103665,0.115091
model_10_6_5,0.808105,-0.159757,-0.410183,-0.205115,0.035899,0.124790,0.104495,0.115239
model_10_6_6,0.808852,-0.156138,-0.419288,-0.206282,0.035760,0.124400,0.105169,0.115351
model_10_6_7,0.809478,-0.152958,-0.426701,-0.207091,0.035642,0.124058,0.105719,0.115428
model_10_6_8,0.809987,-0.150280,-0.432851,-0.207736,0.035547,0.123770,0.106174,0.115490
model_10_6_9,0.810406,-0.147965,-0.437947,-0.208217,0.035469,0.123521,0.106552,0.115536


DataFrame salvo em ../results/metrics_10_6
+++++++++++ [10_7] | 1 ++++++++++++++++++
Elapsed time:  0.43528763299991624
Stopped at epoch:  6
1/1 [==============================] - 0s 24ms/step
+++++++++++ [10_7] | 2 ++++++++++++++++++
Elapsed time:  0.4320301570000993
Stopped at epoch:  6
1/1 [==============================] - 0s 25ms/step
+++++++++++ [10_7] | 3 ++++++++++++++++++
Elapsed time:  0.3441938709997885
Stopped at epoch:  6
1/1 [==============================] - 0s 24ms/step
+++++++++++ [10_7] | 4 ++++++++++++++++++
Elapsed time:  0.30010526900014156
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [10_7] | 5 ++++++++++++++++++
Elapsed time:  0.3966227460000482
Stopped at epoch:  6
1/1 [==============================] - 0s 21ms/step
+++++++++++ [10_7] | 6 ++++++++++++++++++
Elapsed time:  0.5544251910000639
Stopped at epoch:  6
1/1 [==============================] - 0s 24ms/step
+++++++++++ [10_7] | 7 ++++++++++++++++++
Elapsed time:  0.39

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_10_7_2,0.809852,0.667492,0.718879,0.696671,0.035572,0.073047,0.064861,0.069195
model_10_7_1,0.814689,0.719055,0.800420,0.761771,0.034668,0.061719,0.046048,0.054344
model_10_7_0,0.824445,0.838572,0.875398,0.858393,0.032842,0.035463,0.028749,0.032303


DataFrame salvo em ../results/metrics_10_7
+++++++++++ [10_8] | 1 ++++++++++++++++++
Elapsed time:  0.5096747530001267
Stopped at epoch:  6
1/1 [==============================] - 0s 26ms/step
+++++++++++ [10_8] | 2 ++++++++++++++++++
Elapsed time:  1.2203566379998847
Stopped at epoch:  14
1/1 [==============================] - 0s 26ms/step
+++++++++++ [10_8] | 3 ++++++++++++++++++
Elapsed time:  0.546177883999917
Stopped at epoch:  6
1/1 [==============================] - 0s 49ms/step
+++++++++++ [10_8] | 4 ++++++++++++++++++
Elapsed time:  1.1018268060001901
Stopped at epoch:  12
1/1 [==============================] - 0s 30ms/step
+++++++++++ [10_8] | 5 ++++++++++++++++++
Elapsed time:  1.1259523980002086
Stopped at epoch:  11
1/1 [==============================] - 0s 19ms/step
+++++++++++ [10_8] | 6 ++++++++++++++++++
Elapsed time:  0.4672351310000522
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/step
+++++++++++ [10_8] | 7 ++++++++++++++++++
Elapsed time:  0.55